# Import the necessary libraries

In [13]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import numpy as np  # Make sure numpy is imported

# Configure logging
logging.basicConfig(filename='../logs/predict_store_sales.log',
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
  
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from load_data import Load_Data
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


# Load data sets(train and store)

In [14]:
logging.info("Load cleaned store data")

# Create an instance of CSVReader
df = Load_Data('../data/combined_store_data.csv')

# Load the data
df.load_data()

# Get the loaded data
df = df.get_data()


Data successfully loaded from ../data/combined_store_data.csv


# Data overview

In [15]:
print(df.head())

   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      1         c          a               1270.0                        9.0   
2      1         c          a               1270.0                        9.0   
3      1         c          a               1270.0                        9.0   
4      1         c          a               1270.0                        9.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0              NaN              NaN   
1                    2008.0       0              NaN              NaN   
2                    2008.0       0              NaN              NaN   
3                    2008.0       0              NaN              NaN   
4                    2008.0       0              NaN              NaN   

  PromoInterval  DayOfWeek        Date  Sales  Customers  Open  Promo  \
0

# Data Preprocessing

### Handle Categorical Columns

In [4]:


logging.info('Starting preprocessing with updated dataframe')
# One-hot encode StoreType, Assortment, and StateHoliday
df = pd.get_dummies(df, columns=['StoreType', 'Assortment', 'StateHoliday'], drop_first=True)
logging.info('One-hot encoded StoreType, Assortment, and StateHoliday columns')

# Check result
print(df.head())

   Store  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1               1270.0                        9.0   
1      1               1270.0                        9.0   
2      1               1270.0                        9.0   
3      1               1270.0                        9.0   
4      1               1270.0                        9.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0              NaN              NaN   
1                    2008.0       0              NaN              NaN   
2                    2008.0       0              NaN              NaN   
3                    2008.0       0              NaN              NaN   
4                    2008.0       0              NaN              NaN   

  PromoInterval  DayOfWeek        Date  ...  Promo  SchoolHoliday  \
0           NaN          5  2015-07-31  ...      1              1   
1           NaN          4  2015-07-30  ...      1            

###  Handle Competition and Promo2 Features & missing values

In [5]:
logging.info('Handling missing values for Promo2SinceWeek, Promo2SinceYear, and PromoInterval')

# Fill missing values for Promo2SinceWeek and Promo2SinceYear with 0 (no promotion)
df['Promo2SinceWeek'].fillna(0, inplace=True)
df['Promo2SinceYear'].fillna(0, inplace=True)

# Fill missing PromoInterval with 'None'
df['PromoInterval'].fillna('None', inplace=True)
logging.info('Filled missing values in Promo2SinceWeek, Promo2SinceYear, and PromoInterval')

# Check result
print(df[['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].head())


   Promo2SinceWeek  Promo2SinceYear PromoInterval
0              0.0              0.0          None
1              0.0              0.0          None
2              0.0              0.0          None
3              0.0              0.0          None
4              0.0              0.0          None


C:\Users\wubeshet.abera\AppData\Local\Temp\ipykernel_19928\237365783.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Promo2SinceWeek'].fillna(0, inplace=True)
C:\Users\wubeshet.abera\AppData\Local\Temp\ipykernel_19928\237365783.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

### Feature Extraction from Date Column

In [8]:
from datetime import datetime

logging.info('Extracting features from the Date column')

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract weekday
df['DayOfWeek'] = df['Date'].dt.weekday + 1  # Monday=1, Sunday=7

# Extract is_weekend (1 if weekend, 0 if weekday)
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 6 else 0)

# Check if it's beginning, mid, or end of the month
df['MonthDay'] = df['Date'].dt.day
df['IsBeginningOfMonth'] = df['MonthDay'].apply(lambda x: 1 if x <= 10 else 0)
df['IsMidMonth'] = df['MonthDay'].apply(lambda x: 1 if 10 < x <= 20 else 0)
df['IsEndOfMonth'] = df['MonthDay'].apply(lambda x: 1 if x > 20 else 0)

logging.info('Extracted day of week, weekend indicator, and month-day-related features')


### Scale the Data

In [9]:
logging.info('Starting to scale numeric features')

# Define features to scale
features_to_scale = ['Sales', 'CompetitionDistance', 'Promo2SinceWeek', 'Promo2SinceYear']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

logging.info('Scaled features: Sales, CompetitionDistance, Promo2SinceWeek, and Promo2SinceYear')

# Check result
print(df[features_to_scale].head())

      Sales  CompetitionDistance  Promo2SinceWeek  Promo2SinceYear
0 -0.132683            -0.539198        -0.760097        -1.001128
1 -0.195801            -0.539198        -0.760097        -1.001128
2 -0.257620            -0.539198        -0.760097        -1.001128
3 -0.198139            -0.539198        -0.760097        -1.001128
4  0.085244            -0.539198        -0.760097        -1.001128


In [10]:
logging.info('Preprocessing completed and dataset is ready for modeling')

# Display the first few rows to verify
print(df.head())


   Store  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1            -0.539198                        9.0   
1      1            -0.539198                        9.0   
2      1            -0.539198                        9.0   
3      1            -0.539198                        9.0   
4      1            -0.539198                        9.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0        -0.760097        -1.001128   
1                    2008.0       0        -0.760097        -1.001128   
2                    2008.0       0        -0.760097        -1.001128   
3                    2008.0       0        -0.760097        -1.001128   
4                    2008.0       0        -0.760097        -1.001128   

  PromoInterval  DayOfWeek       Date  ...  Assortment_b  Assortment_c  \
0          None          5 2015-07-31  ...         False         False   
1          None          4 2015-07-30  ...         F

# Building Models with Sklearn Pipelines

In [18]:


# Data Preparation: Assuming you have a DataFrame 'df' ready with preprocessed data
# Extract the features and target
X = df.drop(['Sales', 'Date'], axis=1)  # Dropping 'Sales' and 'Date' since Sales is the target
y = df['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Log the shape of the dataset
logging.info(f"Training data shape: {X_train.shape}")
logging.info(f"Testing data shape: {X_test.shape}")

# Define the preprocessing pipeline (e.g., scaling)
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ]
)

# Define the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a pipeline that first applies the preprocessor and then fits the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Log the pipeline setup
logging.info("Pipeline created with StandardScaler and RandomForestRegressor.")

# Fit the model
pipeline.fit(X_train, y_train)

# Log after fitting the model
logging.info("Model training completed.")

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Log prediction completion
logging.info("Model prediction completed.")

# Evaluate the model performance (using RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
logging.info(f"Test RMSE: {rmse:.4f}")

# Log feature importance
feature_importances = model.feature_importances_
important_features = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)
logging.info(f"Feature importances: {important_features}")


ValueError: All arrays must be of the same length